## 特征表分析

## 日期分割

In [171]:
conn = hive.Connection(host='172.30.2.60', port=10000, database='recsys')
train_sql = """select * from recsys.search_full_link_30d limit 1000"""
train_data = pd.read_sql(train_sql, conn)

train_data['search_full_link_30d.log_date'].min()
df3 = train_data.where(train_data['search_full_link_30d.log_date'] <= out_date)

df4 = df3.dropna(how='all')

# 数据准备

In [62]:
from pyhive import hive
import pandas as pd
conn = hive.Connection(host='172.30.2.60', port=10000, database='recsys')

# 搜索日志表

In [26]:
trace_log_sql = """select user_id, resource_id, resource_type, sum(if(action_code=1001,1,0)) as click
from recsys.recommend_event_trace_log where log_date = '2022-11-06' and event_code = '143001' 
group by  user_id, resource_id, resource_type
limit 10"""
trace_log = pd.read_sql(trace_log_sql, conn)

# 物料表

In [32]:
resource_sql = """
select resource_id, resource_type,total_show,total_click,total_share,total_vote,total_collection,total_comment
from recsys.dim_resource where log_date = '2022-11-06'
limit 10"""
resource_log = pd.read_sql(resource_sql, conn)

# 用户表

In [101]:
user_sql = """
select resource_type,sum(total_show)
    from recsys.dim_resource where log_date = '2022-11-06'
    group by resource_type
limit 10"""
user_log = pd.read_sql(user_sql, conn)

In [99]:
user_sql = """
select resource_id, resource_type,total_show,total_click,total_share,total_vote,total_collection,total_comment
    from recsys.dim_resource where log_date = '2022-11-06' and resource_type=''
limit 10"""
user_log = pd.read_sql(user_sql, conn)

# 流量表 left join 物料  left join 用户表

In [85]:

join_sql = """
create table  recsys.search_full_link_30d as
select 
if(trace_log.click_cnt >= 1 ,1,0) as click_label,
trace_log.user_id as user_id, trace_log.resource_id as resource_id, trace_log.resource_type as resource_type,
user_log.sex as sex, user_log.role as role,user_log.age as age,
resource_log.total_show as total_show,
resource_log.total_click as total_click,
resource_log.total_share as total_share,
resource_log.total_vote as total_vote,
resource_log.total_collection as total_collection,
resource_log.total_comment as total_comment,
trace_log.log_date as log_date
from
(
    select log_date, user_id, resource_id, resource_type, sum(if(action_code=1001,1,0)) as click_cnt
    from recsys.recommend_event_trace_log where log_date >= '2022-09-06' and event_code = '143001' 
    group by log_date, user_id, resource_id, resource_type
    
) trace_log
LEFT JOIN 
(
    select resource_id, resource_type,total_show,total_click,total_share,total_vote,total_collection,total_comment
    from recsys.dim_resource where log_date = '2022-11-06'

) resource_log
ON trace_log.resource_id = resource_log.resource_id and trace_log.resource_type = resource_log.resource_type
LEFT JOIN
(
    select id, sex, role, from_unixtime(unix_timestamp(birthday,'yyyyMMdd'),'yyyy-MM-dd')/365 as age 
    from recsys.dim_user where log_date = '2022-11-06'
) user_log
ON trace_log.user_id = user_log.id

"""
join_log = pd.read_sql(join_sql, conn)


TypeError: 'NoneType' object is not iterable

In [44]:
join_log

,trace_log.user_id,trace_log.resource_id,trace_log.resource_type,click_label,resource_log.total_show,resource_log.total_click,resource_log.total_share,resource_log.total_vote,resource_log.total_collection,resource_log.total_comment,user_log.sex,user_log.role
0,211e5d05-acf1-4417-9bc4-644e935672fd,17384,VIDEO,0,221.0,9.0,0.0,2.0,0.0,0.0,None,2
1,211e5d05-acf1-4417-9bc4-644e935672fd,105860700000,ARTICLE,0,NaN,NaN,NaN,NaN,NaN,NaN,None,2
2,211e5d05-acf1-4417-9bc4-644e935672fd,17389,VIDEO,0,3397.0,566.0,0.0,2.0,1.0,0.0,None,2
3,211e5d05-acf1-4417-9bc4-644e935672fd,32524,NEWS,0,318.0,41.0,4.0,9.0,5.0,0.0,None,2
4,211e5d05-acf1-4417-9bc4-644e935672fd,34007,NEWS,0,134.0,18.0,5.0,3.0,2.0,0.0,None,2
...,...,...,...,...,...,...,...,...,...,...,...,...
1995,61a48041e4b0c46dd4c4175b,20211019616ED055E4B04B33C1B57C9D,CASE,0,21.0,6.0,0.0,5.0,0.0,12.0,1,2
1996,61a48041e4b0c46dd4c4175b,5710,ONLINE_ACADEMIC,0,11976.0,28.0,0.0,0.0,0.0,0.0,1,2
1997,61a48041e4b0c46dd4c4175b,201701225884722E0CF28A9589320AB7,CASE,0,9.0,6.0,0.0,2.0,1.0,7.0,1,2
1998,61a48041e4b0c46dd4c4175b,33973,NEWS,0,5482.0,478.0,327.0,18.0,11.0,3.0,1,2


In [69]:
test_sql = """
select * from  recsys.search_full_link_30d limit 10
"""
test_log = pd.read_sql(test_sql, conn)

## Redis

In [123]:
import redis

env = 'dev'

# get environment variable
# env = os.getenv('RECSYS_ENV')
# logger.info("Current environment variable RECSYS_ENV: {env}".format(env=env))
# if env not in ('dev', 'test', 'prod'):
#     raise ValueError('The environment variable RECSYS_ENV should be dev/test/prod, got {env}.'.format(env=env))

# define redis configuration  47.97.207.88 root Oba.2247d
redis_conf = {
    'dev': {'host': '172.29.28.203', 'port': 6379, 'password': None},  # 应用算法开发机
    'test': {'host': '172.16.68.209', 'port': 6379, 'password': '123456'},  # 测试环境
    'prod': {'host': 'r-uf6ojxzvetp1r9m1id.redis.rds.aliyuncs.com', 'port': 6379, 'password': '!@Oba2247d'}  # 阿里云生产环境
}
# define redis connection
host = redis_conf[env]['host']
port = redis_conf[env]['port']
password = redis_conf[env]['password']
r = redis.StrictRedis(host=host, port=port, password=password, decode_responses=True)


In [124]:
r.get

<bound method BasicKeyCommands.get of Redis<ConnectionPool<Connection<host=172.29.28.203,port=6379,db=0>>>>